In [19]:
from langdetect import detect
import numpy as np
import re
import glob
import os
import tqdm
import nltk
nltk.download('punkt') # one time execution

[nltk_data] Downloading package punkt to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
def _getTextFile(langual):
    file_list = glob.glob(f'../data/stopwords/stopwords/*_{langual}.txt')
    files = ",".join(file_list)
    return files

def cleanText(english_txt):
    try:
        word_tokens = english_txt.split()
        filtered_word = [w for w in word_tokens if w not in stop_words and not w.isdigit()]
        filtered_word = [w + " " for w in filtered_word]
        return "".join(filtered_word)
    except:
        return np.nan

def detectLang(txt):
    try:
        return detect(txt)
    except:
        return np.nan

def cleanNonEnglish(txt):
    txt = re.sub(r'\W+', ' ', txt)
    txt = txt.lower()
    txt = txt.replace("[^a-zA-Z]", " ")
    word_tokens = txt.split()
    filtered_word = [w for w in word_tokens if all(ord(c) < 128 for c in w)]
    filtered_word = [w + " " for w in filtered_word]
    return "".join(filtered_word)

def countMinMaxAver(lines):
    min_len = 10000
    aver_len = 0
    max_len = 0
    for temp in lines:
        aver_len = aver_len + len(temp)
        if len(temp) < min_len:
            min_len = len(temp)
        if len(temp) > max_len:
            max_len = len(temp)
    aver_len = 1.0 * aver_len / len(lines)
    print('min_len : ' + str(min_len))
    print('max_len : ' + str(max_len))
    print('average_len : ' + str(aver_len))
    
def titleProcessing(title):
    #title = re.findall(r"title=\"(.*)\">", title)
    #title = title[0].replace(" ", "_")
    return re.findall(r"title=\"(.*)\">", title)

def contentProcessing(content, title_list_i):
    if content[0].strip() == title_list_i[0]:
        return title_list_i, content[1].strip()

In [21]:
stop_words=set()
for file in _getTextFile("en").split(","):
    for word in open(file):
        stop_words.add(word.strip())

In [ ]:
index_content_dic = {}
title_list = []
count = -1
for line in open('./data/wiki.txt'):
    if re.match('<doc id.*>', line):
        title_list.append(titleProcessing(line))
        count+=1
        index_content_dic[count] = []
        continue
    index_content_dic_value = index_content_dic[count]
    index_content_dic_value.append(line)
    index_content_dic[count] = index_content_dic_value
    
countMinMaxAver(index_content_dic.values())

In [ ]:
len(index_content_dic)

In [ ]:
len(title_list)

In [7]:
#key_word = ["film","novel","album","song","band","name","ep","game","surname","tv series"]

In [8]:
title_content_dic = {}
for i, content in tqdm.tqdm(index_content_dic.items()):
    title = title_list[i][0].strip().split()
    if len(title) <= 10 or str(title[1])[0] == "(" and str(title[-1])[-1] == ")":
        #if str(title[-1])[:-1].replace("(","") in key_word:
            #continue
        title, content = contentProcessing(content, title_list[i])
        content = cleanNonEnglish(content).strip()
        content = cleanText(content).strip()
        if len(content) == 0 or title[0].strip()[0] == "(":
            continue
        title_content_dic[title[0]] = content

100%|██████████████████████████████| 6403704/6403704 [04:28<00:00, 23845.46it/s]


In [9]:
len(title_content_dic)

6329832

In [10]:
title_keyword_dict = {}
for k in title_content_dic.keys():
    k = k.strip().split()
    k_title = " ".join(k[1:])
    if k_title not in title_keyword_dict.keys():
        title_keyword_dict[k_title] = 0
        title_keyword_dict[k_title]+=1
    else:
        #title_keyword_dict[k_title].append(k[0])
        title_keyword_dict[k_title]+=1

In [11]:
title_keyword_dict_sorted = sorted(title_keyword_dict.items(), key = lambda kv:(kv[1], kv[0]))

In [12]:
title_keyword_dict_sorted[-11:]

[('language', 6617),
 ('house', 7055),
 ('(album)', 7697),
 ('(film)', 8028),
 ('district', 10178),
 ('river', 10949),
 ('(surname)', 11001),
 ('railway station', 14109),
 ('station', 16103),
 ('(disambiguation)', 29520),
 ('', 805534)]

In [13]:
def removeTitle(title_content_dic):
    title_content = {}
    for k, v in title_content_dic.items():
        if k in v:
            v = v.replace(k, "").strip()
        title_content[k.replace(" ", "_")] = v
    return title_content

In [14]:
wiki_def_data = removeTitle(title_content_dic)

In [15]:
list(wiki_def_data.items())[:10]

[('united_states_v._reynolds',
  'united reynolds landmark legal case formal recognition secrets privilege judicially recognized extension presidential power'),
 ('haya_language',
  'haya oluhaya swahili kihaya bantu language spoken haya people tanzania south southwest coast lake victoria population haya speakers estimated people closest relative nyambo language closely languages southwest uganda nkore kiga rutooro runyoro form group called rutara'),
 ('allana_slater',
  'allana amy slater born april perth western australia retired australian artistic gymnast australian senior national champion multiple medalist commonwealth games slater considered australia internationally successful gymnasts'),
 ('conservation_status',
  'group organisms instance species group exists group extinct future factors account assessing  simply number individuals remaining increase decrease population time breeding success rates threats systems  exist international multi country national local levels consum

In [16]:
np.save("./data/wiki_def_data", title_content_dic)

In [17]:
wiki_def_data["opera"]

'form theatre music fundamental component dramatic roles singers distinct musical theatre work literal translation italian word  typically collaboration composer librettist incorporates number performing arts acting scenery costume dance ballet performance typically  house accompanied orchestra smaller musical ensemble early 19th century led conductor'

In [18]:
wiki_def_data["apple"]

'edible fruit produced  tree malus domestica  trees cultivated worldwide grown species genus malus tree originated central asia wild ancestor malus sieversii today s grown thousands years asia europe brought north america european colonists s religious mythological significance cultures including norse greek european christian tradition'